In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly

import test
import mylayers
import util

# Used to save temporary variables
import pickle

In [2]:
import os
filename = os.path.basename(globals()['__vsc_ipynb_file__']).split('.')[0]
filename

'4_MLP_relu+relu+softmax'

## Read data

In [3]:
df_train = pd.read_csv("./data/fashion-mnist_train.csv")
df_valid = pd.read_csv("./data/fashion-mnist_test.csv")


### Shuffle the data set

In [4]:
df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)


### Seperate feature column and target columns

In [5]:
X_train = np.array(df_train.drop('label', axis=1), dtype="f")
Y_train = df_train.loc[:, 'label']
X_valid = np.array(df_valid.drop('label', axis=1), dtype="f")
Y_valid = df_valid.loc[:, 'label']


### One hot encoding Y

In [6]:
encoder = util.onehot_encoder(Y_train)
Y_train = encoder.encoding(Y_train)
Y_valid = encoder.encoding(Y_valid)


In [7]:
# Testing
# X_train = X_train[:1000]
# Y_train = Y_train[:1000]


## Pretrain

### First FC

In [8]:
X_train_z_scored = mylayers.InputLayer(X_train).forward(X_train)
layers_pre_1 = []
layers_pre_1.append(mylayers.InputLayer(X_train))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(X_train.shape[1], 128)
layers_pre_1.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(128, X_train.shape[1])
layers_pre_1.append(fc_layer)
# ---valid---
layers_pre_1.append(mylayers.SquaredError())

In [9]:
%time
hyperparams = {'epoch': 100, 'eta': 10e-4, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_pre_1,Y_h,_= test.adam_learn(X_train,X_train_z_scored,layers_pre_1,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 100/100 [05:57<00:00,  3.57s/it]


In [10]:
util.draw_scores(ls_scores_pre_1,"None",hyperparams,"SE Score")

In [11]:
h_1 = layers_pre_1[1].getPrevOut()

### Second FC

In [12]:
layers_pre_2 = []
layers_pre_2.append(mylayers.InputLayer(h_1,z_score=False))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(128, 64)
layers_pre_2.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(64, 128)
layers_pre_2.append(fc_layer)
# ---valid---
layers_pre_2.append(mylayers.SquaredError())

In [13]:
%time
hyperparams = {'epoch': 1000, 'eta': 10e-5, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,acc= test.adam_learn(h_1,h_1,layers_pre_2,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 1000/1000 [00:04<00:00, 248.51it/s]


In [14]:
util.draw_scores(ls_scores_train,"None",hyperparams,"SE Score")

In [15]:
h_2 = layers_pre_2[1].getPrevOut()

### Third FC

In [16]:
layers_pre_3 = []
layers_pre_3.append(mylayers.InputLayer(h_2,z_score=False))
# ---In---
fc_layer = mylayers.FullyConnectedLayer(64, 10)
layers_pre_3.append(fc_layer)
# ---Out---
fc_layer = mylayers.FullyConnectedLayer(10, 64)
layers_pre_3.append(fc_layer)
# ---valid---
layers_pre_3.append(mylayers.SquaredError())

In [17]:
%time
hyperparams = {'epoch': 10000, 'eta': 10e-6, 'batch_size':1000, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,acc= test.adam_learn(h_2,h_2,layers_pre_3,mylayers.SquaredError,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 10000/10000 [00:21<00:00, 456.84it/s]


In [18]:
util.draw_scores(ls_scores_train,"None",hyperparams,"SE Score")

In [19]:
h_3 = layers_pre_3[1].getPrevOut()

## Build layers

In [20]:
layers = []
# ---Input Layer---
layers.append(mylayers.InputLayer(X_train))
# ---Hidden Layer---
# Greedy pre trained
fc_layer = layers_pre_1[1]
layers.append(fc_layer)
layers.append(mylayers.ReLuLayer())
# Greedy pre trained
fc_layer = layers_pre_2[1]
layers.append(fc_layer)
layers.append(mylayers.ReLuLayer())
# Greedy pre trained
fc_layer = layers_pre_3[1]
layers.append(fc_layer)
layers.append(mylayers.SoftmaxLayer())
# ---Objective---
layers.append(mylayers.CrossEntropy())


In [21]:
layers

### layers test

In [22]:
%time
hyperparams = {'epoch': 500, 'eta': 10e-6, 'batch_size': 100, 'rho1': 0.9, 'rho2': 0.999}
epoch = hyperparams['epoch']
eta = hyperparams['eta']
batch_size = hyperparams['batch_size']
rho1 = hyperparams['rho1']
rho2 = hyperparams['rho2']
ls_scores_train,Y_hat,ls_accuracy_train,ls_scores_valid,ls_accuracy_valid = test.adam_learn_with_validation(X_train,Y_train,X_valid,Y_valid,layers,mylayers.CrossEntropy,epoch=epoch,eta=eta,print_eval=False,print_layer=False,batch_size=batch_size)


Wall time: 0 ns


100%|██████████| 500/500 [33:52<00:00,  4.06s/it]


In [23]:
train_results = {
    'hyperparams' : hyperparams,
    'ls_scores_train':ls_scores_train,
    'Y_hat':Y_hat,
    'ls_accuracy_train':ls_accuracy_train
    ,'ls_scores_valid':ls_scores_valid,
    'ls_accuracy_valid':ls_accuracy_valid
}

In [24]:
# save model
with open(f"models/model_{filename}.pkl", "wb") as f:
    pickle.dump(layers, f)

# save result to pickle
with open(f"models/{filename}.pkl", "wb") as f:
    pickle.dump(train_results, f)

In [25]:
# read result from pickle
with open(f'models/{filename}.pkl', 'rb') as f:
    train_results = pickle.load(f)
    
hyperparams = train_results['hyperparams']
ls_scores_train = train_results['ls_scores_train']
Y_hat = train_results['Y_hat']
ls_accuracy_train = train_results['ls_accuracy_train']
ls_scores_valid = train_results['ls_scores_valid']
ls_accuracy_valid = train_results['ls_accuracy_valid']

In [26]:
# graph of scores
hidden_layers = "relu -> relu -> softmax"
fig = util.draw_scores_with_validation(
    ls_scores_train, ls_scores_valid, hidden_layers, hyperparams
)
fig.show()


In [27]:
# graph of accuracys
fig = util.draw_accuracys_with_validation(
    ls_accuracy_train, ls_accuracy_valid, hidden_layers, hyperparams
)
fig.show()


### validate accuracy

In [28]:
labeled_Y_hat = encoder.decoding(Y_hat)
labeled_Y_hat


[7,
 8,
 8,
 5,
 9,
 1,
 2,
 0,
 6,
 2,
 5,
 0,
 7,
 1,
 6,
 6,
 6,
 2,
 9,
 1,
 2,
 4,
 8,
 0,
 4,
 9,
 1,
 0,
 0,
 5,
 3,
 6,
 2,
 7,
 3,
 3,
 9,
 9,
 4,
 8,
 2,
 2,
 0,
 2,
 1,
 5,
 4,
 4,
 6,
 9,
 1,
 3,
 2,
 5,
 8,
 9,
 5,
 1,
 1,
 2,
 4,
 8,
 4,
 1,
 2,
 8,
 6,
 9,
 4,
 0,
 5,
 6,
 5,
 6,
 4,
 2,
 0,
 5,
 6,
 1,
 1,
 6,
 9,
 7,
 5,
 2,
 9,
 2,
 6,
 3,
 2,
 4,
 4,
 2,
 9,
 7,
 5,
 8,
 7,
 0,
 8,
 0,
 2,
 7,
 3,
 2,
 1,
 2,
 2,
 4,
 5,
 9,
 2,
 7,
 3,
 1,
 8,
 9,
 1,
 5,
 1,
 3,
 2,
 2,
 7,
 8,
 9,
 5,
 9,
 4,
 5,
 3,
 7,
 1,
 6,
 9,
 2,
 3,
 7,
 3,
 6,
 2,
 9,
 7,
 1,
 1,
 5,
 7,
 8,
 1,
 8,
 5,
 1,
 2,
 2,
 7,
 9,
 0,
 1,
 6,
 9,
 0,
 6,
 1,
 2,
 6,
 5,
 4,
 8,
 3,
 0,
 9,
 3,
 6,
 7,
 5,
 4,
 0,
 8,
 1,
 9,
 9,
 6,
 6,
 9,
 2,
 1,
 6,
 3,
 5,
 5,
 4,
 9,
 1,
 4,
 3,
 1,
 2,
 5,
 2,
 8,
 5,
 1,
 1,
 3,
 9,
 0,
 1,
 0,
 6,
 3,
 8,
 0,
 5,
 8,
 1,
 0,
 9,
 2,
 4,
 7,
 5,
 5,
 9,
 7,
 6,
 6,
 5,
 8,
 4,
 6,
 9,
 2,
 6,
 2,
 3,
 0,
 7,
 9,
 1,
 4,
 2,
 6,
 2,
 7,
 8,
 9,
 7,
 8,
 3,


In [29]:
labeled_Y = encoder.decoding(Y_train)
labeled_Y


[7,
 8,
 8,
 5,
 9,
 1,
 2,
 6,
 6,
 2,
 5,
 0,
 7,
 1,
 6,
 0,
 6,
 2,
 9,
 1,
 2,
 4,
 8,
 0,
 4,
 9,
 1,
 0,
 0,
 5,
 1,
 6,
 2,
 7,
 3,
 4,
 9,
 9,
 4,
 8,
 2,
 6,
 0,
 2,
 1,
 5,
 4,
 4,
 6,
 9,
 1,
 3,
 2,
 5,
 8,
 9,
 5,
 1,
 1,
 2,
 3,
 8,
 3,
 1,
 2,
 8,
 0,
 9,
 4,
 0,
 5,
 6,
 5,
 2,
 6,
 2,
 0,
 5,
 2,
 1,
 1,
 6,
 9,
 7,
 5,
 2,
 9,
 2,
 6,
 3,
 2,
 4,
 4,
 2,
 9,
 7,
 5,
 8,
 7,
 0,
 8,
 0,
 2,
 7,
 3,
 2,
 1,
 2,
 4,
 2,
 5,
 9,
 4,
 7,
 3,
 1,
 8,
 9,
 1,
 5,
 1,
 3,
 2,
 2,
 7,
 8,
 9,
 5,
 9,
 4,
 5,
 3,
 7,
 1,
 6,
 9,
 2,
 3,
 7,
 4,
 6,
 2,
 9,
 7,
 1,
 1,
 5,
 7,
 8,
 1,
 8,
 5,
 1,
 2,
 2,
 7,
 9,
 0,
 1,
 4,
 9,
 0,
 6,
 1,
 2,
 6,
 5,
 4,
 8,
 3,
 0,
 9,
 3,
 6,
 7,
 5,
 4,
 0,
 8,
 1,
 9,
 9,
 6,
 6,
 9,
 2,
 1,
 6,
 3,
 5,
 5,
 4,
 9,
 1,
 4,
 3,
 1,
 2,
 5,
 2,
 8,
 5,
 1,
 1,
 3,
 9,
 0,
 1,
 0,
 6,
 3,
 8,
 0,
 5,
 8,
 1,
 0,
 9,
 2,
 2,
 7,
 5,
 5,
 9,
 7,
 6,
 6,
 5,
 8,
 3,
 6,
 9,
 2,
 6,
 2,
 3,
 0,
 7,
 9,
 1,
 4,
 2,
 6,
 2,
 7,
 8,
 9,
 7,
 8,
 3,


In [30]:
c = 0
for i in range(len(labeled_Y)):
    if labeled_Y[i] == labeled_Y_hat[i]:
        c += 1
print("Final Accuracy:", (c / len(labeled_Y)) * 100, "%")


Final Accuracy: 92.05 %
